In [ ]:
import sys
import os

ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath("")))
sys.path.append(ROOT_DIR)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Scikit-learn utils
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_moons
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc

# Tensorflow imports
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout

# Classifiers for attack models
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier


In [ ]:
#import t=privachy version 
import tensorflow_privacy

from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy


### Definition of the datasets
1. We draw data points from a distribution.
2. We split these data points into the target dataset and a shadow dataset drawn from the same distribution.
3. We also draw a dataset from a different distribution.

**NOTE**. ***I make datasets with few samples but with many features to force the target model to overfit.***


***NOTE JIM: had to make batch_size 25 so DP optimizer would run with same hyperparams

In [ ]:
n_classes = 2

# (X,y): Original distribution
X, y = make_classification(n_samples=1000,
                           n_classes=n_classes, 
                           n_features=300,
                           n_informative=300,
                           n_redundant=0,
                           n_repeated=0,
                           random_state=15
                          )
# One-hot encoding of the label
y = np.eye(n_classes)[y]

# (Xt, yt) is the target dataset, owned by the TRE and drawn from the (X,y) distribution
# (Xs, ys) is a shadow dataset drawn from the (X,y) distribution
Xt, Xs, yt, ys = train_test_split(X, y, test_size=0.50, random_state=15)

# (Xd, yd) is a shadow dataset, drawn from a different distribution (different seed)
Xd, yd = make_classification(n_samples=1000,
                           n_classes=n_classes, 
                           n_features=300,
                           n_informative=300,
                           n_redundant=0,
                           n_repeated=0,
                           random_state=42
                          )
yd = np.eye(n_classes)[yd]




In [ ]:
# Split into train (member) and test (non-member) datasets
# Set shuffle to False so that Xt_membership is consistent with Xt, otherwise
# we need to stack Xt_member and Xt_nonmember again to get a consistent Xt.
Xt_member, Xt_nonmember, yt_member, yt_nonmember = train_test_split(Xt, yt, test_size=0.5, shuffle=False)

# Set membership status for future tests
Xt_membership = np.vstack(
    (
        np.ones((Xt_member.shape[0], 1), np.uint8),
        np.zeros((Xt_nonmember.shape[0], 1), np.uint8)
    )
).flatten()

### Define the target model architecture

*Again, I'm using a rather big model (for the classification task) to favour overfitting.*

In [ ]:
# Define target model
# Tensorflow model (MLP) (making it big to make it overfit)

input_data = Input(shape = Xt_member[0].shape)
x = Dense(128, activation='relu')(input_data)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output = Dense(2, activation='softmax')(x)

## Now try the SafeKerasModel version

In [ ]:
import importlib
import safemodel
from safemodel.classifiers.safekeras import Safe_KerasModel



importlib.reload(safemodel.safemodel)
importlib.reload(safemodel.classifiers.safekeras)
from safemodel.classifiers.safekeras import Safe_KerasModel



In [ ]:
optimizer=None
safeModel = Safe_KerasModel(inputs= input_data, outputs=output,name="safekeras-test")


In [ ]:
#safeModel.__dict__


In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=False, reduction=tf.losses.Reduction.NONE)


safeModel.compile(loss=loss)#optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




In [ ]:
theType= type(safeModel.optimizer)
print(f'optimiser is type {theType}')

dpused,reason = safeModel.check_optimizer_is_DP(safeModel.optimizer)
print(f' It is {dpused} that the model will be DP because {reason}')

In [ ]:
epochs = 20
batch_size = 25

r_DP = safeModel.fit(Xt_member, 
              yt_member, 
              validation_data=(Xt_nonmember, yt_nonmember),
              epochs=epochs, 
              batch_size=batch_size
)  


plt.plot(r_DP.history['accuracy'], label='accuracy')
plt.plot(r_DP.history['val_accuracy'], label='validation accuracy')
plt.legend()
plt.show() 


In [ ]:
theType= type(safeModel.optimizer)
print(f'optimiser is type {theType}')

dpused,reason = safeModel.check_DP_used(safeModel.optimizer)
print(f' It is {dpused} that the model will be DP because {reason}')

## compute privacy

In [ ]:
print(f'dataset has {Xt.shape[0]} entries so batch size is {100*safeModel.batch_size/Xt.shape[0]}%')
privacy = compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=Xt.shape[0],
                                              batch_size=safeModel.batch_size,
                                              noise_multiplier=safeModel.noise_multiplier,
                                              epochs=25,#epochs,
                                              delta=1e-5)
print(f'with these settings privacy = {privacy}')

In [ ]:
safeModel.save('safekeras.sav')

In [ ]:
safeModel.preliminary_check()

In [ ]:
for key,value in safeModel.__dict__.items():
     print (f'thing associated with key {key} has type {type(value)}')

In [ ]:
all_keys = []
unusuals =[]
tuples =[]
deleted_keys = []

print("===============================================================================================")
print("Tuples")
print("===============================================================================================")
for key,value in safeModel.__dict__.items():
    if(type(value) == tuple):
        print (f'thing associated with key {key} has type {type(value)}')
        tuples.append(key)

print("===============================================================================================")
print("Bools")
print("===============================================================================================")
for key,value in safeModel.__dict__.items():
    if(type(value) == bool):
        print (f'thing associated with key {key} has type {type(value)}')

print("===============================================================================================")
print("Lists")
print("===============================================================================================")        
for key,value in safeModel.__dict__.items():
    if(type(value) == list):
        print (f'thing associated with key {key} has type {type(value)}')

print("===============================================================================================")
print("Strings")
print("===============================================================================================")
for key,value in safeModel.__dict__.items():
    if(type(value) == str):
        print (f'thing associated with key {key} has type {type(value)}')

        
print("===============================================================================================")
print("Ints")
print("===============================================================================================")
for key,value in safeModel.__dict__.items():
    if(type(value) == int):
        print (f'thing associated with key {key} has type {type(value)}')

print("===============================================================================================")
print("Floats")
print("===============================================================================================")
for key,value in safeModel.__dict__.items():
    if(type(value) == float):
        print (f'thing associated with key {key} has type {type(value)}')
        
print("===============================================================================================")
print("Dicts")
print("===============================================================================================")
for key,value in safeModel.__dict__.items():
    if(type(value) == dict):
        print (f'thing associated with key {key} has type {type(value)}')

        
print("===============================================================================================")
print("Sets")
print("===============================================================================================")
for key,value in safeModel.__dict__.items():
    if(type(value) == set):
        print (f'thing associated with key {key} has type {type(value)}')

print("===============================================================================================")
print("Unusual Ones")
print("===============================================================================================")

        
for key,value in safeModel.__dict__.items():
    if((type(value) != int) and (type(value) != str) and 
       (type(value) != list) and (type(value) != bool) and 
       (type(value) != tuple) and (type(value) != float) and
       (type(value) != dict) and (type(value) != set)):
        print (f'thing associated with key {key} has type {type(value)}')
        unusuals.append(key)

In [ ]:
#safeModel.request_release('safekeras.pkl') # TypeError: cannot pickle '_thread.RLock' object

#safeModel.request_release('safekeras.sav') # TypeError: cannot pickle '_thread.RLock' object

#safeModel.request_release('safekeras.tf')




safeModel.save('my_model')
safeModel.save_weights('weights.h5')

safeModel.request_release('safe.sav')

In [ ]:


safeModel.request_release('safekeras.h5')

In [ ]:
importlib.reload(safemodel.safemodel)
importlib.reload(safemodel.classifiers.safekeras)
from safemodel.classifiers.safekeras import Safe_KerasModel

optimizer=None
safeModel = Safe_KerasModel(inputs= input_data, outputs=output,name="safekeras-test")

loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=False, reduction=tf.losses.Reduction.NONE)


safeModel.compile(loss=loss)#optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 20
batch_size = 25

r_DP = safeModel.fit(Xt_member, 
              yt_member, 
              validation_data=(Xt_nonmember, yt_nonmember),
              epochs=epochs, 
              batch_size=batch_size
)  


# Testing
## Test 1 - researcher doesn't change recommended params

In [ ]:
importlib.reload(safemodel.safemodel)
importlib.reload(safemodel.classifiers.safekeras)
from safemodel.classifiers.safekeras import Safe_KerasModel


# create and fit using recommended params


print("***Test 1: researcher doesn't change recommended params")
optimizer=None
safeKerasModel1 = Safe_KerasModel(inputs=input_data, outputs=output,name="safekerasmodel1-test")
safeKerasModel1.compile(loss=loss)
safeKerasModel1.fit(Xt_member, 
                    yt_member, 
                    validation_data=(Xt_nonmember, yt_nonmember),
                    epochs=epochs, 
                    batch_size=batch_size)
safeKerasModel1.save("safe1.h5", include_optimizer=False, save_format='h5')
safeKerasModel1.preliminary_check()
safeKerasModel1.request_release(filename="safe1.h5")

## Test 2 - researcher changes params safely
Inputs: 

l2_norm_clip = 1.1, noise_multiplier=0.6

Expected Result:

Remain Unchanged l2_norm_clip = 1.1, noise_multiplier=0.6

In [ ]:
importlib.reload(safemodel.safemodel)
importlib.reload(safemodel.classifiers.safekeras)
from safemodel.classifiers.safekeras import Safe_KerasModel

# change model params to recommended values
print("\n***Test 2: researcher changes params safely")
safeKerasModel2 = Safe_KerasModel(inputs= input_data, outputs=output,name="safekerasmodel2-test", l2_norm_clip = 1.1, noise_multiplier=0.6)
safeKerasModel2.optimizer=tensorflow_privacy.DPKerasAdamOptimizer
safeKerasModel2.compile(loss=loss)
safeKerasModel2.l2_norm_clip = 1.1
safeKerasModel2.noise_multiplier = 0.6
safeKerasModel2.fit(Xt_member, 
                    yt_member, 
                    validation_data=(Xt_nonmember, yt_nonmember),
                    epochs=epochs, 
                    batch_size=batch_size)

safeKerasModel2.save("safe2.hd5")
safeKerasModel2.preliminary_check()
safeKerasModel2.request_release(filename="safe2.hd5")

## Test 3 - researcher changes string params unsafely
Expected result:

WARNING: model parameters may present a disclosure risk:
- parameter l2_norm_clip = 0.8 identified as less than the recommended min value of 1.0.
Changed parameter l2_norm_clip = 1.0.


In [ ]:
importlib.reload(safemodel.safemodel)
importlib.reload(safemodel.classifiers.safekeras)
from safemodel.classifiers.safekeras import Safe_KerasModel

# change one model params in an unsafe way
print("\n***Test 3: researcher changes params unsafely")
safeKerasModel3 = Safe_KerasModel(inputs= input_data, outputs=output,name="safekerasmodel3-test", l2_norm_clip = 0.8)
safeKerasModel3.compile(loss=loss)
#safeKerasModel3.optimizer="Adam"
safeKerasModel3.fit(Xt_member, 
                    yt_member, 
                    validation_data=(Xt_nonmember, yt_nonmember),
                    epochs=epochs, 
                    batch_size=batch_size)
safeKerasModel3.save("unsafe3.hd5")
safeKerasModel3.preliminary_check()
safeKerasModel3.request_release(filename="unsafe3.hd5")

## Test 3 - researcher changes string params unsafely
Input delta = -1 

Expected Result:

WARNING: model parameters may present a disclosure risk:
- parameter delta = -1 identified as less than the recommended min value of 1e-05.
Changed parameter delta = 1e-05.

In [ ]:
# change another model params in an  unsafe way
importlib.reload(safemodel.safemodel)
importlib.reload(safemodel.classifiers.safekeras)
from safemodel.classifiers.safekeras import Safe_KerasModel

print("\n***Test 3: researcher changes string params unsafely")
safeKerasModel4 = Safe_KerasModel(inputs= input_data, outputs=output,name="safekerasmodel3-test", delta=-1)
safeKerasModel4.compile(loss=loss)
#safeKerasModel3.optimizer="Adam"
safeKerasModel4.fit(Xt_member, 
                    yt_member, 
                    validation_data=(Xt_nonmember, yt_nonmember),
                    epochs=epochs, 
                    batch_size=batch_size)
safeKerasModel4.save("unsafe3.hd5")
safeKerasModel4.preliminary_check()
safeKerasModel4.request_release(filename="unsafe3.hd5")

## Test 5: researcher changes params unsafely
input: l2_norm_clip=0.9, noise_multipier=0.4, min_epsilon=9,delta=-1

Expected Result:

WARNING: model parameters may present a disclosure risk:
- parameter l2_norm_clip = 0.9 identified as less than the recommended min value of 1.0.
Changed parameter l2_norm_clip = 1.0.
- parameter min_epsilon = 9 identified as less than the recommended min value of 10.
Changed parameter min_epsilon = 10.
- parameter delta = -1 identified as less than the recommended min value of 1e-05.
Changed parameter delta = 1e-05.

In [ ]:
# change another model params in an  unsafe way
print("\n***Test 5: researcher changes string and numeric params unsafely")

importlib.reload(safemodel.safemodel)
importlib.reload(safemodel.classifiers.safekeras)
from safemodel.classifiers.safekeras import Safe_KerasModel


safeKerasModel4 = Safe_KerasModel(inputs= input_data, outputs=output,name="safekerasmodel3-test", l2_norm_clip=0.9, noise_multipier=0.4, min_epsilon=9,delta=-1)
safeKerasModel4.compile(loss=loss)
safeKerasModel4.fit(Xt_member, 
                    yt_member, 
                    validation_data=(Xt_nonmember, yt_nonmember),
                    epochs=epochs, 
                    batch_size=batch_size)
safeKerasModel4.save("unsafe4.hd5")
safeKerasModel4.preliminary_check()
safeKerasModel4.request_release(filename="unsafe4.hd5")

## Test 6 researcher changes params unsafely

Expected result:

Input optimizer = wobble

WARNING: model parameters may present a disclosure risk
Unknown optimizer wobble - Changed parameter optimizer = 'DPKerasSGDOptimizer'

In [ ]:
importlib.reload(safemodel.safemodel)
importlib.reload(safemodel.classifiers.safekeras)
from safemodel.classifiers.safekeras import Safe_KerasModel

# change another model params in an  unsafe way but tells preliminary_check() not to overwrite params
print("\n***Test 6: researcher changes string and numeric params unsafely")
safeKerasModel6 = Safe_KerasModel(inputs= input_data, outputs=output,name="safekerasmodel6-test", optimizer="wobble")
safeKerasModel6.compile(loss=loss)
safeKerasModel6.fit(Xt_member, 
                    yt_member, 
                    validation_data=(Xt_nonmember, yt_nonmember),
                    epochs=epochs, 
                    batch_size=batch_size)
safeKerasModel6.save("unsafe6.hd5")

safeKerasModel6.preliminary_check(apply_constraints=False)
safeKerasModel6.request_release(filename="unsafe6.hd5")

## Test 7 researcher changes params unsafely

Inputs:

Input optimizer = Adam

Expected Result:

WARNING: model parameters may present a disclosure risk
Changed parameter optimizer = 'DPKerasAdamOptimizer'

In [ ]:
importlib.reload(safemodel.safemodel)
importlib.reload(safemodel.classifiers.safekeras)
from safemodel.classifiers.safekeras import Safe_KerasModel

# change another model params in an  unsafe way but tells preliminary_check() not to overwrite params
print("\n***Test 7: researcher changes string and numeric params unsafely")
safeKerasModel7 = Safe_KerasModel(inputs= input_data, outputs=output,name="safekerasmodel6-test", optimizer="Adam")
safeKerasModel7.compile(loss=loss)
safeKerasModel7.fit(Xt_member, 
                    yt_member, 
                    validation_data=(Xt_nonmember, yt_nonmember),
                    epochs=epochs, 
                    batch_size=batch_size)
safeKerasModel7.save("unsafe7.hd5")

safeKerasModel7.preliminary_check(apply_constraints=False)
safeKerasModel7.request_release(filename="unsafe7.hd5")

### Dictionary Tests

In [ ]:
import h5py
importlib.reload(safemodel.safemodel)
importlib.reload(safemodel.classifiers.safekeras)
from safemodel.classifiers.safekeras import Safe_KerasModel

# change another model params in an  unsafe way but tells preliminary_check() not to overwrite params
print("\n***Test 8: researcher changes string and numeric params unsafely")
safeKerasModel8 = Safe_KerasModel(inputs= input_data, outputs=output,name="safekerasmodel6-test", optimizer="Adam")
safeKerasModel8.compile(loss=loss)
safeKerasModel8.fit(Xt_member, 
                    yt_member, 
                    validation_data=(Xt_nonmember, yt_nonmember),
                    epochs=epochs, 
                    batch_size=batch_size)
safeKerasModel8.save("unsafe8.h5", include_optimizer=False, save_format='h5')

print("Now Load the file")

safeKerasModel8 = safeKerasModel8.load('unsafe8.h5')

print('loaded file')



In [ ]:
importlib.reload(safemodel.safemodel)
importlib.reload(safemodel.classifiers.safekeras)
from safemodel.classifiers.safekeras import Safe_KerasModel
f = tf.keras.models.load_model('unsafe8.h5', custom_objects={"Safe_KerasModel": Safe_KerasModel})
kerasmodelkeys =[]
for key,value in f.__dict__.items():
        #print ( f' key {key} : value {value} \n')
        kerasmodelkeys.append(key)

        
ourmodelkeys =[]
for key,value in safeKerasModel8.__dict__.items():
        #print ( f' key {key} : value {value} \n')
        ourmodelkeys.append(key)

print(f'Our model has {len(ourmodelkeys)} keys')
print(f'The keras model has {len(kerasmodelkeys)} keys')
ourmodelkeys.sort()
kerasmodelkeys.sort()

print(ourmodelkeys)

move_list = []

for key,value in f.__dict__.items():
        #print ( f' key {key} : value {value} \n')
        if(key in f.ignore_items):
            print ( f' key {key} is in ignore_list: value {value} \n')
            print ( f' key {key} is type {type(key)}\n')
            if(type(key) == type(move_list)):
              move_list.append(key)
            
        else:
            pass
print('Keys to move to examine_seperately')
print(move_list)


    


In [ ]:
for key,value in safeKerasModel8.__dict__.items():
    if 'optimizer' in key:
        print ( f' key {key} : value {value} \n')

## dp_epsilon_met


In [ ]:

import pandas as pd
importlib.reload(safemodel.safemodel)
importlib.reload(safemodel.classifiers.safekeras)
from safemodel.classifiers.safekeras import Safe_KerasModel

samples = [250, 250, 250]
batch_sizes = [1, 5, 25]
epochs = [20, 200, 2500]
noisemult = [0.7, 0.8, 0.9]

okslist = []
epsilonslist = []
noisemultslist = []
sampleslist = []
batch_size_list = []
epochs_list = []

safeKerasModel8 = Safe_KerasModel(inputs= input_data, outputs=output,name="safekerasmodel6-test", optimizer="Adam")

for i in range(0,3):
    for j in range(0,3):
        for k in range(0,3):
            sampleslist.append(samples[i])
            safeKerasModel8.noise_multiplier = noisemult[i]
            noisemultslist.append(noisemult[i])
            batch_size_list.append(batch_sizes[j])
            epochs_list.append(epochs[k])
            ok, epsilon = safeKerasModel8.dp_epsilon_met(num_examples=samples[i],batch_size=batch_sizes[j],epochs=epochs[k])
            okslist.append(ok)
            epsilonslist.append(epsilon)

for i in range(0, len(okslist)):
    pass
    #print(f'{okslist[i]} {epsilonslist[i]} {noisemultslist[i]} {sampleslist[i]} {epochs_list[i]}' )
    

    
mydf = pd.DataFrame(
    {'OK': okslist,
     'Epsilon': epsilonslist,
     'Noise Multiplier': noisemultslist,
     'Num Samples': sampleslist,
     'Epochs': epochs_list,
     'Batch Size': batch_size_list


    })

mydf.round(decimals=2).sort_values('Epsilon')


### Meets DP Epsilon Criteria

In [ ]:
importlib.reload(safemodel.safemodel)
importlib.reload(safemodel.classifiers.safekeras)
from safemodel.classifiers.safekeras import Safe_KerasModel

epochs=20
batch_size=1
# change another model params in an  unsafe way i.e. Optimizer=Adam
print("\n***Test 9: Meets DP epsilon criteria")
safeKerasModel9 = Safe_KerasModel(inputs= input_data, outputs=output,name="safekerasmodel9-test", optimizer="Adam")
safeKerasModel9.compile(loss=loss)
safeKerasModel9.fit(Xt_member, 
                    yt_member, 
                    validation_data=(Xt_nonmember, yt_nonmember),
                    epochs=epochs, 
                    batch_size=batch_size)
safeKerasModel9.save("unsafe7.hd5")

safeKerasModel9.preliminary_check(apply_constraints=False)
safeKerasModel9.request_release(filename="unsafe9.hd5")

### Test 10 Optimizer is set as DP Version from the start

In [ ]:
import tensorflow_privacy as tf_privacy
importlib.reload(safemodel.safemodel)
importlib.reload(safemodel.classifiers.safekeras)
from safemodel.classifiers.safekeras import Safe_KerasModel

epochs=20
batch_size=1
# change another model params in an  safe way i.e. Optimizer=tf_privacy.DPKerasAdamOptimizer
print("\n***Test 10: Optimizer is set as DP version from the start")
safeKerasModel10 = Safe_KerasModel(inputs= input_data, outputs=output,name="safekerasmodel10-test", optimizer=tf_privacy.DPKerasAdamOptimizer)
safeKerasModel10.compile(loss=loss)
safeKerasModel10.fit(Xt_member, 
                    yt_member, 
                    validation_data=(Xt_nonmember, yt_nonmember),
                    epochs=epochs, 
                    batch_size=batch_size)
safeKerasModel10.save("unsafe7.hd5")

safeKerasModel10.preliminary_check(apply_constraints=False)
safeKerasModel10.request_release(filename="unsafe10.hd5")